In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [26]:
# typical neural network:
# - Define the neural network that has some learnable parameters (or weights)
# - Iterate over a dataset of inputs
# - Process input through the network
# - Compute the loss (how far is the output from being correct)
# - Propagate gradients back into the network’s parameters
# - Update the weights of the network, typically using a simple update rule: weight = weight - learning_rate * gradient

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # 1 input channel (image)
        # 6 output channels
        # 3x3 square convolution
        
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        # affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120) # 6*6 image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # max pooling over (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # single number if size is square
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except batch
        num_features = 1
        
        for s in size:
            num_features *= s
        
        return num_features
    
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [27]:
# learnable parameters
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1 weight

10
torch.Size([6, 1, 3, 3])


In [28]:
# TEST: random 32x32 input
input_ = torch.randn(1, 1, 32, 32)
out = net(input_)
print(out)

tensor([[ 0.0896,  0.0326, -0.0190,  0.0262,  0.1696,  0.0447,  0.0526, -0.0142,
         -0.1228,  0.0500]], grad_fn=<AddmmBackward>)


In [37]:
# zero gradient buffers and backprop
#net.zero_grad()
#out.backward(torch.randn(1, 10))

In [38]:
# optimization
import torch.optim as optim

# create optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)
optimizer.zero_grad()

In [39]:
# loss function
output = net(input_)
target = torch.randn(10) # dummy target
target = target.view(1, -1) # reshape
criterion = nn.MSELoss() # mean-squared error between input and target

loss = criterion(output, target)
print(loss)
print(loss.grad_fn) # MSELoss
print(loss.grad_fn.next_functions[0][0]) # linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0]) # relu

tensor(0.8113, grad_fn=<MseLossBackward>)


In [40]:
# backprop
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()
optimizer.step()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0127, -0.0067,  0.0029, -0.0229, -0.0094,  0.0023])


In [41]:
# update weights: weight = weight - learning_rate * gradient
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)